In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [5]:
import pandas as pd
import numpy as np
from datetime import datetime
import pandas as pd
import torch
import torch.nn as nn

pd.set_option('display.max_columns', None)
np.set_printoptions(precision=15)

root_dir = "/Users/trevorwiebe/Ktor/radar_backend/radar_data/"

data = pd.read_csv(root_dir + 'data/combined_data.csv')

In [6]:
# Setting device
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [7]:
# prepare data frame to be dateTime | latitude | longitude | reflectivity | reflectivity_1 to _15
from copy import deepcopy as dc

def prepare_dataframe_for_lstm(df, n_steps):
  df = dc(df)

  df['datetime'] = pd.to_datetime(df['dateTime'])

  # Encode 'datetime' as cyclical features (excluding day encoding)
  df['minute_sin'] = np.sin(2 * np.pi * df['datetime'].dt.minute / 60)
  df['minute_cos'] = np.cos(2 * np.pi * df['datetime'].dt.minute / 60)
  df['hour_sin'] = np.sin(2 * np.pi * df['datetime'].dt.hour / 24)
  df['hour_cos'] = np.cos(2 * np.pi * df['datetime'].dt.hour / 24)
  df['month_sin'] = np.sin(2 * np.pi * df['datetime'].dt.month / 12)
  df['month_cos'] = np.cos(2 * np.pi * df['datetime'].dt.month / 12)

  # Move new columns to the front
  new_columns = ['minute_sin', 'minute_cos', 'hour_sin', 'hour_cos', 'month_sin', 'month_cos']
  remaining_columns = [col for col in df.columns if col not in new_columns]
  df = df[new_columns + remaining_columns]

  df = df.drop(columns=['dateTime'])
  df = df.drop(columns=['datetime'])

  for i in range(1, n_steps+1):
    df[f'reflectivity_{i}'] = df['reflectivity'].shift(i)

  df.dropna(inplace=True)

  df = df[['reflectivity'] + [col for col in df.columns if col != 'reflectivity']]

  return df

lookback = 15
df = prepare_dataframe_for_lstm(data, lookback)

In [8]:
# remove rows where all -99
# List of the columns reflectivity_1 to reflectivity_15
reflectivity_columns = [f'reflectivity_{i}' for i in range(1, 16)]
reflectivity_columns.append('reflectivity')

# convert all the rows that are less than 0.0 to 0.0
df[reflectivity_columns] = df[reflectivity_columns].mask(df[reflectivity_columns] <= 0, 0)

# Remove rows where all values in the reflectivity columns are 0.0
no_zero_df = df[~(df[reflectivity_columns] == 0.0).all(axis=1)]

In [9]:
#Split into X and y, and shape correctly
X_df = no_zero_df.iloc[:, 7:]
y_df = no_zero_df.iloc[:, 0]

lat_lon = X_df.iloc[:, [0,1]].to_numpy()
time_series_data = X_df.iloc[:, 2:].to_numpy()
time_series_data = time_series_data.reshape(X_df.shape[0], 15, 1)
time_series_data[time_series_data < 15] = 15
lat_lon_repeated = np.repeat(lat_lon[:, np.newaxis, :], 15, axis=1)

X = np.concatenate([lat_lon_repeated, time_series_data], axis=2)
y = y_df.to_numpy().flatten()

In [ ]:
X[:1]

In [ ]:
# Create train, val and test splits

train_split = int(X.shape[0] * .8)
val_split = int(X.shape[0] * .9)

X_train, y_train = X[:train_split], y[:train_split]
X_val, y_val = X[train_split:val_split], y[train_split:val_split]
X_test, y_test = X[val_split:], y[val_split:]

X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

# # Use this if starting from scratch
# model1 = Sequential()
# model1.add(InputLayer((lookback, 3)))
# # LSTM layer with 64 units and dropout for regularization
# model1.add(LSTM(64, return_sequences=False))  # return_sequences=False because we predict one value
# model1.add(Dropout(0.2))  # Helps prevent overfitting

# # Dense layer for additional feature extraction
# model1.add(Dense(32, activation='relu'))  # Increased neurons for more complexity
# model1.add(Dropout(0.2))  # More dropout

# # Final output layer (predicting a single value)
# model1.add(Dense(1, activation='linear'))

# model1.summary()

In [9]:
model1 = load_model(root_dir + 'model/model4.keras')
cp = ModelCheckpoint(root_dir + 'model/model4.keras', save_best_only=True)
model1.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])

In [ ]:
model1.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=4, callbacks=[cp])

In [119]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_colwidth', None)

In [ ]:
print("Finished training at " + datetime.now().strftime('%d/%m/%y %H:%M:%S.%f'))
test_predictions = model1.predict(X_test).flatten()
third_column = np.flip(X_test[:, :, X_test.shape[2]-1], axis=1)
X_test_strings = [' '.join(map(str, row)) for row in third_column]
test_results = pd.DataFrame(data={'Historical':X_test_strings, 'Actuals':y_test, 'Val Predictions':test_predictions,})
sorted_results = test_results.sort_values(by='Actuals', ascending=False)
sorted_results[:1000]